graph db
conjuctive regaulr path queires poptimization


Cardinality estimation

https://15799.courses.cs.cmu.edu/spring2025/notes/07-joins1.pdf
https://15799.courses.cs.cmu.edu/spring2025/schedule.html
https://www.microsoft.com/en-us/research/publication/extensible-query-optimizers-in-practice/ book

starburst
volcano 
cascades

DPhyp
enumerate all
hypergraph



https://arxiv.org/pdf/2407.04823 Path-based Algebraic Foundations of Graph Query Languages
https://openproceedings.org/2025/conf/edbt/paper-234.pdf Path-based Algebraic Foundations of Graph Query Languages

https://dl.acm.org/doi/abs/10.1145/3725237 Minimizing Conjunctive Regular Path Queries

https://en.wikipedia.org/wiki/Regular_path_query
https://homepages.inf.ed.ac.uk/libkin/papers/icdt12b.pdf

networkx?
sqlite?

SQL/PCQ
GQL https://en.wikipedia.org/wiki/Graph_Query_Language

RPQ regular expressions as queries

I have wanted recursive queries for embedding egraphs in graph rewriting. share*  and E*

datalog could do it by materializing the needed transitive closure?
https://docs.google.com/document/d/1eq7jED7XMnOuq80PggkVKtqWTL1wQ8SdZMIWHSZLoPM/edit?tab=t.0

filtyer push down is smaller
hjow to use cost model

https://rustmagazine.org/issue-2/write-a-sql-optimizer-using-egg/

basic graph patterns

sqlite = bad?
It does suck encoding into it

paths asd first class

GPC - graph pattern calculus

cq conjunctive query
rpq - recursive path query
two way rpq

https://github.com/egraphs-good/egg/discussions/189

lexicographic path ordering

select(join(A,B)) -> join(select(A), select(B))
We make it bigger.

cross > join > select

associativity join(A, join(B,C,keys1),keys2) = join(join(A,B, keys3), C, keys4)

"proof relevant" associatvity?

commutavity join(A,B,keys) = join(B,A,keys)

Maybe join with no keys.
And rename nodes?
Just set expressions? Union, Inter, Select, 


https://arxiv.org/pdf/2202.10390

datalog is common 




stefania is doing kat? So is that a useful model?

Optimized kleene algerba regex expresions
Model checking using graph databases



In [ ]:
class App():
    f : str
    args : tuple["App", ...]
    kwargs : dict[str, "App"]




In [ ]:
from dataclasses import dataclass
class Rel(): ...

class Join():
    A : Rel
    B : Rel
    cols : list[str]

class Union():
    A : Rel
    B : Rel

class Star():
    R : Rel

class Proj(): # Select, Sigma
    R : Rel
    cond : ?


SyntaxError: invalid syntax (498765439.py, line 17)

In [ ]:
class Proj():
    cols : set[str]
    R : Rel
class Join():
    A : Rel
    B : Rel
    on : list[tuple[str, str]] # (A.col, B.col)
class Filter():
    vs : dict[str, any] # col -> value
    R : Rel



In [ ]:
@dataclass
class App():
    f : str
    args : tuple['App', ...] = ()
proj()

def rw():
    ...


In [ ]:

class GraphDB():
    vertices : set[object]
    edges: 


 rdflib https://rdflib.readthedocs.io/en/stable/gettingstarted/#a-sparql-query-example

In [ ]:
%%file /tmp/rel.egg

(datatype Query
    (Join Query Query) ; (Vec String)?
    (Union Query Query)
    (Star Query)
    ()
)



In [ ]:
import egglog
E = egglog.EGraph()

class Math(egglog.Expr):



In [3]:
dir(egglog)

['Action',
 'BackOff',
 'BaseExpr',
 'BigInt',
 'BigIntLike',
 'BigRat',
 'BigRatLike',
 'Bool',
 'BoolLike',
 'BuiltinExpr',
 'Command',
 'Container',
 'ConvertError',
 'CostModel',
 'EGraph',
 'EggSmolError',
 'Expr',
 'ExprCallable',
 'ExprValueError',
 'Fact',
 'GraphvizKwargs',
 'GreedyDagCost',
 'Map',
 'MapLike',
 'MultiSet',
 'Primitive',
 'PyObject',
 'Rational',
 'RewriteOrRule',
 'Ruleset',
 'Schedule',
 'Set',
 'SetLike',
 'StageInfo',
 'String',
 'StringLike',
 'TimeOnly',
 'Unit',
 'UnstableFn',
 'Vec',
 'VecLike',
 'WithPlan',
 '_BirewriteBuilder',
 '_EqBuilder',
 '_NeBuilder',
 '_RewriteBuilder',
 '_SetBuilder',
 '_UnionBuilder',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'back_off',
 'bindings',
 'birewrite',
 'builtins',
 'check',
 'check_eq',
 'config',
 'constant',
 'conversion',
 'convert',
 'converter',
 'declarations',
 'deconstruct',
 'default_cost_model',
 'define_expr_method',

In [1]:
%%file /tmp/query.ml


type constr = Eq of string * string

type query = Select of string * query | Join of query * query
  ;;


let opt1 (q : query) = match q with
  | Select (col, Join (q1, q2)) -> Some (col, q1, q2)
  | _ -> None
;;


print_endline "Hello, Pathquery!"

Writing /tmp/query.ml


In [2]:
! ocaml /tmp/query.ml

Hello, Pathquery!
